In [8]:
# basic
import json
import pandas as pd
from tqdm import tqdm
import tiktoken

# import langchain
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chains import ConversationChain

# Vector Store
from qdrant_client import QdrantClient
from langchain.vectorstores import Qdrant
from langchain.vectorstores.pgvector import PGVector

# import env
from config import Settings
settings = Settings()

In [9]:
# jsonファイルの読み込み
def load_json():
    with open("../data/questions.json", "r") as f:
        query_json = json.load(f)
    return query_json

In [34]:
# モデルの初期化
def chain(system_role_prompt: str, user_prompt: str) -> str:
    chat = AzureChatOpenAI(
        temperature=0.9,
        openai_api_key=settings.AZURE_OPENAI_API_KEY,
        openai_api_base=settings.AZURE_OPENAI_API_ENDPOINT,
        deployment_name=settings.AZURE_OPENAI_API_DEPLOYMENT_NAME,
        openai_api_version=settings.AZURE_OPENAI_API_VERSION,
        openai_api_type="azure",
        max_tokens=100,
    )

    chat_prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(system_role_prompt),
        HumanMessagePromptTemplate.from_template('{input}')
    ])

    chain = ConversationChain(
        llm=chat,
        prompt=chat_prompt,
    )

    response = chain.predict(user_prompt)

    return response

In [35]:
# トークン数の取得
def token_encoder_initialize() -> tiktoken.Encoding:
    return tiktoken.get_encoding("cl100k_base")

In [37]:
chain('', 'こんにちは')

ValidationError: 1 validation error for ConversationChain
__root__
  Got unexpected prompt input variables. The prompt expects ['input'], but got ['history'] as inputs from memory, and input as the normal input key. (type=value_error)